In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.preprocessing as pre
import tensorflow as tf
import pandas as pd

In [3]:
df = pd.read_csv("Training_Datasets/clickbait_dataset.csv", sep = ";;", header = None) 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
df

,0,1
0,Clip-On Man Buns Are Here And They Absolutely ...,1
1,We Know What Cat GIF Will Make You Smile Based...,1
2,25 Gifts For The Future Revolutionary In Your ...,1
3,"The Fish Are Biting, and the Room Is Hopping",0
4,Kindle Joins a Bookstore Ritual: Authors Can S...,0
5,"Deadly crash at Dayton, Ohio air show",0
6,Afghans riot after civilians die in crash,0
7,26 Questions Cats Just Need To Answer,1
8,28 Kids Whose Futures Burn As Bright As The Sun,1
9,"A Ruling Against Intel, Unlikely to Drag It Down",0


In [3]:
sentences = list(df[0])
labels = list(df[1])

In [4]:
NTRAIN = 25000

tfidf = TfidfVectorizer(min_df=2)
x_train = np.array(tfidf.fit_transform(sentences[:NTRAIN]).todense(), dtype=np.float32)
x_validation  = np.array(tfidf.transform    (sentences[NTRAIN:]).todense(), dtype=np.float32)

x_train = pre.normalize(x_train)
x_validation  = pre.normalize(x_validation)

y_train = np.array([int(i) for i in labels[:NTRAIN]])
y_validation = np.array([int(i) for i in labels[NTRAIN:]])

In [15]:
x_validation.shape

(6898, 11198)

In [5]:
nsamples = x_train.shape[0]
dim      = x_train.shape[1]


In [ ]:
om

In [13]:
W

<tf.Variable 'Variable:0' shape=(11198, 20) dtype=float32_ref>

In [22]:
# Parameters
learning_rate = 0.1
training_epochs = 30
display_step = 1
hidden = 50 

# tf Graph Input: 3 variables: query, pos and negative. 
qry = tf.placeholder(tf.float32, [1, dim])
pos = tf.placeholder(tf.float32, [1, dim])
neg = tf.placeholder(tf.float32, [1, dim])

# The embedding parameters
# Projection matrix
W = tf.Variable(tf.random_normal([dim, hidden]))
# We also add a bias term
b = tf.Variable(tf.zeros([hidden]))

# Function to embed the inputs
eqry = tf.matmul(qry, W) + b
epos = tf.matmul(pos, W) + b
eneg = tf.matmul(neg, W) + b

# Define triplet loss in the space of the embeddings
cost = tf.maximum(0.0, 1 + tf.matmul(eqry, tf.transpose(tf.subtract(eneg, epos))))

# Gradient Descent Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Function to initialize the variables
init = tf.global_variables_initializer()

In [23]:
from tqdm import tqdm_notebook as t

In [24]:
# Launch the graph
with tf.Session() as sess:
    with tf.device("/cpu:0"):
        # Initialize
        sess.run(init)
        # Training cycle
        for epoch in t(range(training_epochs)):
            avg_cost = 0.
            # Loop over all samples
            for i in t(np.arange(0, nsamples)):        

                # Each "input sample" consists of a "query", a positive and a negative example
                # This is constructed on the fly
                # Ideally, this should be precomputed

                # Query: take the next training example
                x_qry = x_train[i,None]

                # Positive: sample a training example from the same class as the query
                idx  = np.where(y_train == y_train[i])[0]
                r = np.random.randint(len(idx))
                ipos = idx[r]
                x_pos = x_train[ipos,None]

                # Negative: sample a training example with a different class label 
                idx  = np.where(np.logical_not(y_train == y_train[i]))[0]
                r = np.random.randint(len(idx))
                ineg = idx[r]
                x_neg = x_train[ineg,None]

                # Perform gradient step
                _, c = sess.run([optimizer, cost], feed_dict={qry: x_qry,
                                                              pos: x_pos,
                                                              neg: x_neg})
                # Update loss of this epoch
                avg_cost += c 

            # Display logs per epoch step
            if (epoch+1) % display_step == 0:
                print("Epoch:", epoch+1, avg_cost/nsamples)
                What = sess.run(W)
                bhat = sess.run(b)
                #eval_metric(What, bhat, epoch+1)

        print("Optimization Finished!")

        # Persist model parameters as numpy arrays
        What = sess.run(W)
        bhat = sess.run(b)


HBox(children=(IntProgress(value=0, max=30), HTML(value='')))

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



HBox(children=(IntProgress(value=0, max=25000), HTML(value='')))

Epoch: 1 [[ 2.04458141]]


HBox(children=(IntProgress(value=0, max=25000), HTML(value='')))

...

Epoch: 28 [[ 0.02197287]]


HBox(children=(IntProgress(value=0, max=25000), HTML(value='')))

Epoch: 29 [[ 0.02417287]]


HBox(children=(IntProgress(value=0, max=25000), HTML(value='')))

Epoch: 30 [[ 0.02193732]]

Optimization Finished!


In [30]:
What

array([[ 0.88582551,  1.78828537, -0.345101  , ...,  0.87472552,
        -2.16043806,  0.0350652 ],
       [ 1.09983814,  0.65346789, -1.02648914, ...,  1.11845434,
        -1.53929853,  2.03804636],
       [ 1.25184262,  1.19486451, -0.87867063, ..., -1.28952682,
         1.02735925,  0.95543367],
       ..., 
       [-1.31413233,  0.39289135, -0.18674383, ..., -0.49574944,
         0.33873397,  0.00872943],
       [ 0.69012976,  0.42899829,  0.83736825, ..., -0.47003588,
        -2.27261162, -1.35065055],
       [ 0.55077803, -1.63117433, -1.07352388, ...,  0.47789291,
        -0.03499657, -0.98743045]], dtype=float32)

In [31]:
bhat

array([-0.1756824 , -0.45093077,  0.25746205,  0.79566044,  0.06249893,
        0.65364933,  0.45786107,  1.69509935, -0.11413354,  0.483235  ,
       -0.86550969, -0.5786044 ,  0.6043914 , -0.33628663,  0.35213807,
       -0.7548874 , -0.25672656,  1.22718978, -0.79815543, -0.16066757,
        0.06709519,  1.27700377, -0.04564873, -0.01235188, -0.30198997,
        0.24242461,  0.31654716, -0.17957844, -0.32362896, -0.03640467,
       -0.37408477, -0.98614597, -1.22010064,  0.06457223,  0.60204887,
       -1.30006933, -1.31264389, -0.24843091,  0.6883713 , -1.47674751,
        1.6934998 , -0.59668386,  0.63307655,  0.18150443,  0.85031796,
        0.55460316,  0.72789311,  1.36535776, -0.16906667, -0.47566187], dtype=float32)

In [34]:
def embed(vec):
    return vec@What + bhat

In [54]:
y_train.shape

(25000,)

In [57]:
y_validation.shape

(6898,)

In [61]:
xk = np.vstack([embed(x_train), embed(x_validation)])
yk = np.vstack([y_train.reshape(len(y_train),1), y_validation.reshape(len(y_validation), 1)])

In [63]:
yk.shape

(31898, 1)

In [50]:
xk_classifier.shape

(25000, 50)

In [51]:
xk_val.shape

(6898, 50)

In [78]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(13, input_dim=50, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(xk, yk, validation_split=0.2, epochs=25, batch_size=10)

Train on 25518 samples, validate on 6380 samples
Epoch 1/25
25518/25518 [==============================] - 3s 101us/step - loss: 0.0214 - acc: 0.9948 - val_loss: 0.1427 - val_acc: 0.9707
Epoch 2/25
25518/25518 [==============================] - 2s 91us/step - loss: 0.0045 - acc: 0.9991 - val_loss: 0.1478 - val_acc: 0.9704
Epoch 3/25
25518/25518 [==============================] - 2s 93us/step - loss: 0.0039 - acc: 0.9993 - val_loss: 0.1486 - val_acc: 0.9702
Epoch 4/25
25518/25518 [==============================] - 2s 93us/step - loss: 0.0034 - acc: 0.9993 - val_loss: 0.1555 - val_acc: 0.9707
Epoch 5/25
25518/25518 [==============================] - 2s 92us/step - loss: 0.0031 - acc: 0.9993 - val_loss: 0.1620 - val_acc: 0.9702
Epoch 6/25
25518/25518 [==============================] - 2s 90us/step - loss: 0.0028 - acc: 0.9993 - val_loss: 0.1668 - val_acc: 0.9707
Epoch 7/25
25518/25518 [==============================] - 2s 92us/step - loss: 0.0026 - acc: 0.9994 - val_loss: 0.1723 - val_acc

KeyboardInterrupt: 

In [84]:
NTRAIN = 25000
>>> from sklearn.feature_extraction.text import CountVectorizer

tfidf = CountVectorizer(min_df=2)
x_train = np.array(tfidf.fit_transform(sentences[:NTRAIN]).todense(), dtype=np.float32)
x_validation  = np.array(tfidf.transform    (sentences[NTRAIN:]).todense(), dtype=np.float32)

x_train = pre.normalize(x_train)
x_validation  = pre.normalize(x_validation)

y_train = np.array([int(i) for i in labels[:NTRAIN]])
y_validation = np.array([int(i) for i in labels[NTRAIN:]])

In [87]:
from keras.models import Sequential
from keras.layers import Dense

model2 = Sequential()
model2.add(Dense(13, input_dim=11198, activation='relu'))
model2.add(Dense(5000, activation='relu'))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
# Compile model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model2.fit(x_train, y_train, validation_split=0.2, epochs=10, batch_size=10)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 11s 569us/step - loss: 0.1236 - acc: 0.9509 - val_loss: 0.0774 - val_acc: 0.9736
Epoch 2/10
20000/20000 [==============================] - 10s 517us/step - loss: 0.0299 - acc: 0.9904 - val_loss: 0.1093 - val_acc: 0.9628
Epoch 3/10
20000/20000 [==============================] - 10s 522us/step - loss: 0.0119 - acc: 0.9957 - val_loss: 0.1374 - val_acc: 0.9710
Epoch 4/10
20000/20000 [==============================] - 11s 528us/step - loss: 0.0056 - acc: 0.9982 - val_loss: 0.1525 - val_acc: 0.9712
Epoch 5/10
20000/20000 [==============================] - 10s 499us/step - loss: 0.0026 - acc: 0.9994 - val_loss: 0.1826 - val_acc: 0.9690
Epoch 6/10
20000/20000 [==============================] - 11s 535us/step - loss: 0.0022 - acc: 0.9994 - val_loss: 0.1759 - val_acc: 0.9688
Epoch 7/10
20000/20000 [==============================] - 10s 498us/step - loss: 0.0017 - acc: 0.9994 - val_loss: 0.2